Author: Diederik Jilderda

Date: 02-11-2021

This is the script used to run the CARETHY+ game. Required modules are Pandas and OS to read and write data from Excel files, Numpy and Topogenesis to perform array functions and describe neighbourhoods, and Pygame for the visual implementation of the game for a better user experience. 

Pygame is a rather unstable module. Every time you want to run the game, you have to reload the notebook because it very regularly does not accept new requests or properly loads fonts when you attempt to play the game a second time. There is not much we can do about it, sorry. I added a reflection where I recommend how the game can be further developed. 

In [1]:
import topogenesis as tg
import numpy as np
import pandas as pd 
import pygame as pg 
import os 
pg.font.init()
pg.init()

pygame 2.0.2 (SDL 2.0.16, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

The building blocks of the game are spaces (rooms, courtyards, circulation, etc.) which possess a multitude of properties. These properties are stored within class objects. We did the same for zones and Pygame buttons. All properties are managed in an Excel file and can be altered there. 

In [2]:
class space():
    def __init__(self, name, tag, legend, type, id, shape_x, shape_y, connection, color_id, floor_id, floor_color_id):
        self.name = name
        self.tag = tag
        self.legend = legend
        self.type = type 
        self.id = id
        self.array = np.full((shape_x, shape_y),self.id)
        self.shape = self.array.shape
        self.connection = connection 
        self.color_id = color_id
        self.floor_id = floor_id
        self.floor_color_id = floor_color_id
       

class zone(): 
    def __init__(self, name, build_order): 
        self.name = name 
        self.build_order = build_order


class button(): 
    def __init__(self,color,x,y,width,height,active=False,text=''): 
        self.color = mycolors[color] 
        self.x = x 
        self.y = y 
        self.width = width
        self.height = height 
        self.active = active 
        self.text = text 
    
    def draw(self,window): 
        if self.active == False: 
            color = mycolors[0]
        elif self.active == True: 
            color = self.color
        
        pg.draw.rect(window, color, (self.x,self.y,self.width,self.height),0)

        if self.text != '': 
            font = pg.font.SysFont('lucidaconsole',10)
            text = font.render(self.text, 1, text_color)
            window.blit(text, (self.x + (self.width//2 - text.get_width()//2), self.y + (self.height//2 - text.get_height()//2)))

    def isOver(self,pos): 
        if pos[0] > self.x and pos[0] < self.x + self.width: 
            if pos[1] > self.y and pos[1] < self.y + self.height:
                return True 
        return False 

Here the various properties of objects are read from Excel using Pandas and loaded into their relevant classes. The objects are stored as key-object pairs in dictionaries so they can be called easily. 

In [3]:
mycolors = {}
df_colors = pd.read_excel(os.path.join('Data', '1_Configuration_Data.xlsx'), sheet_name='colors')
for k,v in df_colors.iterrows(): 
    mycolors[v[0]] = v[1], v[2], v[3]


mybuttons = {}
button_list = []
df_buttons = pd.read_excel(os.path.join('Data', '1_Configuration_Data.xlsx'), sheet_name='buttons')
for k,v in df_buttons.iterrows(): 
    mybuttons[v[0]] = button(v[1],v[2],v[3],v[4],v[5],v[6],v[7])
    button_list.append(v[0])


myspaces = {}
circulation_options = []
courtyard_options = []
df_spaces = pd.read_excel(os.path.join('Data', '1_Configuration_Data.xlsx'), sheet_name='spaces')
for k,v in df_spaces.iterrows(): 
    myspaces[v[0]] = space(v[0],v[1],v[2],v[3],v[4],v[5],v[6],v[7],v[8],v[9],v[10])
    if v[3] == 'courtyard': 
        courtyard_options.append(myspaces[v[0]])
    if v[3] == 'circulation': 
        circulation_options.append(myspaces[v[0]])


zone_list = ['general', 'emergency', 'outpatient', 'inpatient', 'administration']
zone_order = []
for i in zone_list: 
    build_order = []
    df_zone = pd.read_excel(os.path.join('Data', '1_Configuration_Data.xlsx'), sheet_name=f'zone_{i}')
    for k,v in df_zone.iterrows():
        build_order.extend([myspaces[v[0]]] * v[1])
    zone_order.append(zone(i,build_order))


Now that all data is loaded, it is time to initialize the game grid. Von Neumann stencils are set up to describe the 1-step orthogonal neighbourhoods of all cells on the lattice. We use three lattices per floor to track what is happening on each floor: lattice_floor to place room ID's, lattice_floor_index to associate the find_neighbours function to the lattice, and finally lattice_floor_color so Pygame can assign colors to the cells. 

We set up a bounding box on the edges of the lattice so that the find_neighbours function does not wrap around the lattice. Finally, we add streets to the lattice. 

For the first floor, we pretend that initially the entire floor is a boundary unit, so all cells are occupied. These are replaced by unoccupied cells as the game progresses on the ground floor, but since no room has a connection ID associated to the boundary cell ID it is not possible to create rooms out-of-the-blue despite there being available space. The only way to reach the first floor is by building a staircase first. 

In [4]:
#von neumann stencil 
stencil = tg.create_stencil("von_neumann",1)
stencil[:,1,1]=0

#empty ground floor lattices 
lattice_cols = 47
lattice_rows = 32
lattice_bg = tg.lattice([[0, 0, 0], [0, lattice_rows-1, lattice_cols-1]], default_value=0, dtype=int)
lattice_bg_index = lattice_bg.indices
lattice_bg_color = tg.lattice([[0, 0, 0], [0, lattice_rows-1, lattice_cols-1]], default_value=0, dtype=int)

# empty first floor lattices 
lattice_f1 = tg.lattice([[0, 0, 0], [0, lattice_rows-1, lattice_cols-1]], default_value=myspaces['boundary'].id, dtype=int)
lattice_f1_index = lattice_f1.indices
lattice_f1_color = tg.lattice([[0, 0, 0], [0, lattice_rows-1, lattice_cols-1]], default_value=myspaces['street'].color_id, dtype=int)


#set up boundary and streets
lattice_bg[0,-3,:12] = myspaces['street'].id
lattice_bg[0,-5,11:] = myspaces['street'].id
lattice_bg[0,-4,11] = myspaces['street'].id
lattice_bg[0,:,2] = myspaces['street'].id 
lattice_bg[0,5,2:14] = myspaces['street'].id
lattice_bg[0,-5:,33] = myspaces['street'].id
lattice_bg[0,:6,13] = myspaces['street'].id

lattice_bg_color[0,-3,:12] = myspaces['street'].color_id
lattice_bg_color[0,-5,11:] = myspaces['street'].color_id
lattice_bg_color[0,-4,11] = myspaces['street'].color_id
lattice_bg_color[0,:,2] = myspaces['street'].color_id
lattice_bg_color[0,5,2:14] = myspaces['street'].color_id
lattice_bg_color[0,-5:,33] = myspaces['street'].color_id
lattice_bg_color[0,:6,13] = myspaces['street'].color_id

lattice_bg[0,:,0] = myspaces['boundary'].id
lattice_bg[0,0,:] = myspaces['boundary'].id
lattice_bg[0,:,-1] = myspaces['boundary'].id
lattice_bg[0,-1,:] = myspaces['boundary'].id

lattice_bg_color[0,:,0] = myspaces['boundary'].color_id
lattice_bg_color[0,0,:] = myspaces['boundary'].color_id
lattice_bg_color[0,:,-1] = myspaces['boundary'].color_id
lattice_bg_color[0,-1,:] = myspaces['boundary'].color_id

#set up unavailable first floor
lattice_f1_color[0,:,0] = myspaces['boundary'].color_id
lattice_f1_color[0,0,:] = myspaces['boundary'].color_id
lattice_f1_color[0,:,-1] = myspaces['boundary'].color_id
lattice_f1_color[0,-1,:] = myspaces['boundary'].color_id

The build function is the base tool to play the game. It requires input for which room to build, its orientation and direction, and its starting cell. It checks if any of the cells are already occupied, and does nothing if this is indeed the case. 

Successfully built rooms are added to both the ID and color lattices. Built modules on ground floor that are allowed to have a stacked module on top of them (1x1, 1x2 and 2x2 shapes) also add unoccupied values (0) to the first floor lattice, essentially freeing up buildable space on the first floor. 

Courtyards are automatically built with a surrounding riwaq (circulation space). Unfortunately it is not possible to align a courtyard centered (so expanding in both directions from starting cell), I would recommend this as an improvement to the functionality of the script. 

When a room from the build order is successfully built, it is removed from the queue. When the build order queue is empty, the active zone is removed from the zone list. Obviously, when there are no zones left to complete the game is finished. Congratulations in advance! 

In [5]:
def build(room, orientation, direction_ns, direction_we, i_selected, j_selected):
    index = lattice_bg_index[0, i_selected, j_selected]

    #orientation 
    if orientation == 'landscape': 
        oriented_room = room.array 
        x = oriented_room.shape[0]
        y = oriented_room.shape[1]
    elif orientation == 'portrait': 
        oriented_room = np.ndarray.transpose(room.array)
        x = oriented_room.shape[0]
        y = oriented_room.shape[1]
    
    #direction 
    if direction_ns == 'up': 
        u = -1
        v = 0
    elif direction_ns == 'down': 
        u = 0
        v = 1
    if direction_we == 'left': 
        q = -1
        r = 0
    elif direction_we == 'right': 
        q = 0
        r = 1
    
    #when the active lattice is the ground floor 
    if mybuttons['button_bg'].active == True: 
        ##place room based on inputs 
        #check if any cells are already occupied, return if any occupancies exist 
        position = np.where(lattice_bg_index == index)
        if np.any(lattice_bg[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] != 0): 
            text_notification = 'Collision! One or more cells are already occupied, try again'
            return 
        else: 
            ##add the room ID to ground floor lattice, also add the floor ID (usually either boundary, or unoccupied) to the first floor lattices to clear up space 
            #the complicated equation is necessary to support Python's syntax: when performing functions over a range of indices, the index range has to be noted as i:j where j>i
            #the u,v,q,r coefficients take care of this difficulty and make it possible to freely choose the desired orientation and direction
            lattice_bg[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = room.id  
            lattice_bg_color[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = room.color_id
            lattice_f1[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = room.floor_id  
            lattice_f1_color[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = room.floor_color_id

            #add room borders to a list which will be drawn later, no borders around circulation/courtyard/environment 
            if room.type != 'circulation' and room.type != 'courtyard' and room.type != 'environment':
                border = pg.Rect((int(position[2])+q*(int(y)-1)) * cell_size + window_offset_x, (int(position[1])+u*(int(x-1))) * cell_size + window_offset_y, y*cell_size, x*cell_size)
                border_bg_list.append(border)

            #add legend to a list which will be drawn later 
            if room.type != 'circulation' and room.type != 'environment':
                legend_surface = legend_font.render(room.legend, 1, text_color)
                legend_bg_text.append(legend_surface)
                legend_bg_x.append( ((int(position[2])+q*(int(y)-1)) * cell_size + window_offset_x) + ( y*cell_size - legend_surface.get_width())//2 )
                legend_bg_y.append( ((int(position[1])+u*(int(x-1))) * cell_size + window_offset_y) + ( x*cell_size - legend_surface.get_height())//2 )

            #staircases are built on both floors simultaneously, add the border and legend to the draw lists of the first floor 
            if room.type == 'staircase': 
                border = pg.Rect((int(position[2])+q*(int(y)-1)) * cell_size + window_offset_x, (int(position[1])+u*(int(x-1))) * cell_size + window_offset_y, y*cell_size, x*cell_size)
                border_f1_list.append(border)
                legend_f1_text.append(legend_surface)
                legend_f1_x.append( ((int(position[2])+q*(int(y)-1)) * cell_size + window_offset_x) + ( y*cell_size - legend_surface.get_width())//2 )
                legend_f1_y.append( ((int(position[1])+u*(int(x-1))) * cell_size + window_offset_y) + ( x*cell_size - legend_surface.get_height())//2 )

            #courtyards automatically come with a surrounding riwaq, so first we fill the entire shape with circulation and then carve out the center with courtyard 
            if room.type == 'courtyard': 
                lattice_bg[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = circulation_options[0].id 
                lattice_bg[0 , int(position[1])+u*(int(x-1)) + 1 : int(position[1])+v*int(x)-u - 1, int(position[2])+q*(int(y)-1) + 1 : int(position[2])+r*int(y)-q - 1 ] = room.id 
                lattice_bg_color[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = circulation_options[0].color_id 
                lattice_bg_color[0 , int(position[1])+u*(int(x-1)) + 1 : int(position[1])+v*int(x)-u - 1, int(position[2])+q*(int(y)-1) + 1 : int(position[2])+r*int(y)-q - 1 ] = room.color_id
                lattice_f1[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = circulation_options[0].floor_id 
                lattice_f1[0 , int(position[1])+u*(int(x-1)) + 1 : int(position[1])+v*int(x)-u - 1, int(position[2])+q*(int(y)-1) + 1 : int(position[2])+r*int(y)-q - 1 ] = room.floor_id 
                lattice_f1_color[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = circulation_options[0].floor_color_id 
                lattice_f1_color[0 , int(position[1])+u*(int(x-1)) + 1 : int(position[1])+v*int(x)-u - 1, int(position[2])+q*(int(y)-1) + 1 : int(position[2])+r*int(y)-q - 1 ] = room.floor_color_id

            #rooms from the build order queue are deleted here, same goes for zone list management 
            if room == zone_order[0].build_order[0]: 
                del zone_order[0].build_order[0]
                if len(zone_order[0].build_order) == 0: 
                    text_notification = 'Congratulations, you completed the zone!'
                    del circulation_options[0]
                    del zone_order[0]
                    if len(zone_order) == 0:
                        text_notification = 'Congratulations, you completed the CARETHY+ game!'
                    return 

    #when the active lattice is the first floor
    if mybuttons['button_f1'].active == True: 
        #staircases can't be built on the first floor because we have no second floor 
        if room.type != 'staircase': 
            ##place room based on inputs 
            #check if any cells are already occupied, return if any occupancies exist
            position = np.where(lattice_f1_index == index)
            if np.any(lattice_f1[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] != 0): 
                text_notification = 'Collision! One or more cells are already occupied, try again'
                return 
            else: 
                #first floor rooms are independent from ground floor so we can just add their room IDs to the first floor lattices 
                lattice_f1[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = room.id  
                lattice_f1_color[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = room.color_id

                #add room borders to a list which will be drawn later, no borders around circulation/courtyard/environment 
                if room.type != 'circulation' and room.type != 'courtyard' and room.type != 'environment':
                    border = pg.Rect((int(position[2])+q*(int(y)-1)) * cell_size + window_offset_x, (int(position[1])+u*(int(x-1))) * cell_size + window_offset_y, y*cell_size, x*cell_size)
                    border_f1_list.append(border)

                #add legend to a list which will be drawn later 
                if room.type != 'circulation' and room.type != 'environment':
                    legend_surface = legend_font.render(room.legend, 1, text_color)
                    legend_f1_text.append(legend_surface)
                    legend_f1_x.append( ((int(position[2])+q*(int(y)-1)) * cell_size + window_offset_x) + ( y*cell_size - legend_surface.get_width())//2 )
                    legend_f1_y.append( ((int(position[1])+u*(int(x-1))) * cell_size + window_offset_y) + ( x*cell_size - legend_surface.get_height())//2 )

                #it is unlikely that enough space exists to build a courtyard on the first floor, but it is possible (of course pavement, not vegetative)
                if room.type == 'courtyard': 
                    lattice_f1[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = circulation_options[0].id 
                    lattice_f1[0 , int(position[1])+u*(int(x-1)) + 1 : int(position[1])+v*int(x)-u - 1, int(position[2])+q*(int(y)-1) + 1 : int(position[2])+r*int(y)-q - 1 ] = room.id 
                    lattice_f1_color[0 , int(position[1])+u*(int(x-1)) : int(position[1])+v*int(x)-u , int(position[2])+q*(int(y)-1) : int(position[2])+r*int(y)-q] = circulation_options[0].color_id 
                    lattice_f1_color[0 , int(position[1])+u*(int(x-1)) + 1 : int(position[1])+v*int(x)-u - 1, int(position[2])+q*(int(y)-1) + 1 : int(position[2])+r*int(y)-q - 1 ] = room.color_id

                #rooms from the build order queue are deleted here, same goes for zone list management 
                if room == zone_order[0].build_order[0]: 
                    del zone_order[0].build_order[0]
                    if len(zone_order[0].build_order) == 0: 
                        text_notification = 'Congratulations, you completed the zone!'
                        del circulation_options[0]
                        del zone_order[0]
                        if len(zone_order) == 0:
                            text_notification = 'Congratulations, you completed the CARETHY+ game!'
                        return text_notification



The draw function draws all elements on the screen. Pygame draws elements on top of each other in the order of their code lines. 
We define a variable called 'active_lattice' to indicate which floor is selected and thus which cells to draw on the screen. 

In [6]:
def draw_window(active_lattice, active_lattice_color, text_notification): 
    #add background 
    window.fill(mycolors[3])

    #draw the lattice cells based on which lattice is active 
    for i in range(active_lattice_color.shape[1]): 
        for j in range(active_lattice_color.shape[2]): 
            color = mycolors[active_lattice_color[0,i,j]]    
            infill = pg.Rect(window_offset_x+cell_size*j,window_offset_y+cell_size*i,cell_size,cell_size)
            pg.draw.rect(window,color,infill)

    #draw notification bar  
    textbox = pg.Rect(200,750,475,25)
    pg.draw.rect(window,mycolors[0],textbox)
    notification_font = pg.font.SysFont('lucidaconsole', 12)
    text_notification_display = notification_font.render(text_notification, 1, text_color)
    window.blit(text_notification_display,(207,757))

    #adds a white background for the sidebar on the right 
    info_background = pg.Rect(1000,25,500,800)
    pg.draw.rect(window,mycolors[0],info_background)

    #these elements all form the sidebar together 
    img_logo = pg.image.load(os.path.join('Sprites', 'Carethy_logo.png'))
    img_logo_scaled = pg.transform.scale(img_logo, (450,100))
    window.blit(img_logo_scaled,(1025,35))

    img_rules = pg.image.load(os.path.join('Sprites', 'Carethy_rules.png'))
    img_rules_scaled = pg.transform.scale(img_rules, (450,575))

    img_legend = pg.image.load(os.path.join('Sprites', 'Carethy_legend.png'))
    img_legend_scaled = pg.transform.scale(img_legend, (450,575))
    
    img_zone_general = pg.image.load(os.path.join('Sprites', 'Carethy_zones_general.png'))
    img_zone_general_scaled = pg.transform.scale(img_zone_general, (450,575))
    img_zone_emergency = pg.image.load(os.path.join('Sprites', 'Carethy_zones_emergency.png'))
    img_zone_emergency_scaled = pg.transform.scale(img_zone_emergency, (450,575))
    img_zone_outpatient = pg.image.load(os.path.join('Sprites', 'Carethy_zones_outpatient.png'))
    img_zone_outpatient_scaled = pg.transform.scale(img_zone_outpatient, (450,575))
    img_zone_inpatient = pg.image.load(os.path.join('Sprites', 'Carethy_zones_inpatient.png'))
    img_zone_inpatient_scaled = pg.transform.scale(img_zone_inpatient, (450,575))
    img_zone_administration = pg.image.load(os.path.join('Sprites', 'Carethy_zones_administration.png'))
    img_zone_administration_scaled = pg.transform.scale(img_zone_administration, (450,575))

    if mybuttons['button_rules'].active == True: 
        img_sidebar = img_rules_scaled

    if mybuttons['button_zones'].active == True: 
        if len(zone_order) != 0: 
            if zone_order[0].name == 'general': 
                img_sidebar = img_zone_general_scaled
            if zone_order[0].name == 'emergency': 
                img_sidebar = img_zone_emergency_scaled
            if zone_order[0].name == 'outpatient': 
                img_sidebar = img_zone_outpatient_scaled
            if zone_order[0].name == 'inpatient': 
                img_sidebar = img_zone_inpatient_scaled
            if zone_order[0].name == 'administration': 
                img_sidebar = img_zone_administration_scaled
        else: 
            img_sidebar = img_legend_scaled 

    if mybuttons['button_legend'].active == True: 
        img_sidebar = img_legend_scaled

    window.blit(img_sidebar,(1025,200))

    for button in button_list: 
        mybuttons[button].draw(window)

    #based on which floor is active, draw all respective room borders and legends 
    if mybuttons['button_bg'].active == True: 
        for border in border_bg_list: 
            pg.draw.rect(window,mycolors[3],border,2)
        for i in range(len(legend_bg_text)): 
            window.blit(legend_bg_data[0][i],(legend_bg_data[1][i],legend_bg_data[2][i]))
    if mybuttons['button_f1'].active == True: 
        for border in border_f1_list: 
            pg.draw.rect(window,mycolors[3],border,2) 
        for i in range(len(legend_f1_text)): 
            window.blit(legend_f1_data[0][i],(legend_f1_data[1][i],legend_f1_data[2][i]))

    pg.display.update()

Here we create empty lists which will be used as containers to add data like room borders and legends. 

In [7]:
border_bg_list = []
border_f1_list = []
output_bg = []
output_f1 = []
legend_bg_text = []
legend_bg_x = []
legend_bg_y = []
legend_bg_data = [legend_bg_text, legend_bg_x, legend_bg_y]
legend_f1_text = []
legend_f1_x = []
legend_f1_y = []
legend_f1_data = [legend_f1_text, legend_f1_x, legend_f1_y]

The main function is used to play the game. The function requires a continuous time loop to run, any interruptions of this (like errors) make it crash. 
We set up the last variables required to play the game. 

The main function mostly does three things: keep track of events, keep track of the status of buttons, and determine which cells are viable. Nested in this function are the draw and build functions.  

In [8]:
window_offset_x = 30
window_offset_y = 105
window_width = 1500 
window_height = 800
window = pg.display.set_mode((window_width,window_height))
pg.display.set_caption('CARETHY+ GAME')
fps = 60
cell_size = 20 
line_width = 1
legend_font = pg.font.SysFont('lucidaconsole', 8)
text_color = mycolors[3]


def main(): 
    run = True 
    clock = pg.time.Clock()
  
    text_notification = 'Welcome to the Carethy Game!'

    courtyard_choice = 0
    i_selected = None  
    j_selected = None 
    active_lattice = lattice_bg
    active_lattice_color = lattice_bg_color
    active_lattice_index = lattice_bg_index

    screenshot_number = 1

    #keeping this while loop running is imperative, otherwise the window freezes 
    while run: 
        clock.tick(fps)

        for event in pg.event.get(): 
            if event.type == pg.QUIT: 
                run = False 

            #keep track of when a button is clicked and which button should respond to this through cursor location 
            #many buttons are connected in the sense that if one is 'on', the other(s) should be 'off'
            if event.type == pg.MOUSEBUTTONDOWN:
                pos = pg.mouse.get_pos() 
                if mybuttons['button_screenshot'].isOver(pos):
                    rect_subwindow = pg.Rect(window_offset_x + cell_size,window_offset_y + cell_size,(lattice_cols-2) * cell_size,(lattice_rows-2) * cell_size)
                    subwindow = window.subsurface(rect_subwindow)
                    pg.image.save(subwindow, os.path.join('Screenshots',f'Carethy_Screenshot_{screenshot_number}.jpg'))
                    screenshot_number += 1

                if mybuttons['button_bg'].isOver(pos):
                    i_selected = None 
                    j_selected = None
                    mybuttons['button_bg'].active = True 
                    mybuttons['button_f1'].active = False 
                if mybuttons['button_f1'].isOver(pos):
                    i_selected = None 
                    j_selected = None
                    mybuttons['button_bg'].active = False
                    mybuttons['button_f1'].active = True   

                if mybuttons['button_rules'].isOver(pos):
                    mybuttons['button_rules'].active = True 
                    mybuttons['button_zones'].active = False 
                    mybuttons['button_legend'].active = False
                if mybuttons['button_zones'].isOver(pos):
                    mybuttons['button_rules'].active = False 
                    mybuttons['button_zones'].active = True 
                    mybuttons['button_legend'].active = False
                if mybuttons['button_legend'].isOver(pos):
                    mybuttons['button_rules'].active = False 
                    mybuttons['button_zones'].active = False 
                    mybuttons['button_legend'].active = True

            #if the zone list is empty, the game crashes because certain parts of the game will ask for a list that doesn't exist
            #this if-statement prevents the game from crashing by isolating these parts and ignore them in case there are no zones left 
            if len(zone_order) != 0: 
                if event.type == pg.MOUSEBUTTONDOWN:
                    pos = pg.mouse.get_pos() 

                    #if player clicks within the grid, then the corresponding cell is selected 
                    if pos[0] >= window_offset_x and pos[0] <= window_offset_x + lattice_cols * cell_size and pos[1] >= window_offset_y and pos[1] <= window_offset_y + lattice_rows * cell_size: 
                        j_selected = int((pos[0] - window_offset_x) // (cell_size)) 
                        i_selected = int((pos[1] - window_offset_y) // (cell_size))
                        index = active_lattice_index[0, i_selected, j_selected]
                        index_coord = np.where(active_lattice_index == index)

                        #these three lines are crucial, they disallow the player to select any nonviable cell 
                        if np.all(viable != index): 
                            i_selected = None 
                            j_selected = None

                    #here a bunch of connected buttons are programmed to interact 
                    if mybuttons['button_landscape'].isOver(pos):
                        mybuttons['button_landscape'].active = True 
                        mybuttons['button_portrait'].active = False 
                    if mybuttons['button_portrait'].isOver(pos):
                        mybuttons['button_portrait'].active = True 
                        mybuttons['button_landscape'].active = False 

                    if mybuttons['button_up'].isOver(pos):
                        mybuttons['button_up'].active = True 
                        mybuttons['button_down'].active = False 
                    if mybuttons['button_down'].isOver(pos):
                        mybuttons['button_down'].active = True 
                        mybuttons['button_up'].active = False 

                    if mybuttons['button_right'].isOver(pos):
                        mybuttons['button_right'].active = True 
                        mybuttons['button_left'].active = False 
                    if mybuttons['button_left'].isOver(pos):
                        mybuttons['button_left'].active = True 
                        mybuttons['button_right'].active = False 
                    
                    #reset the index when a new space is selected, because a viable cell for one space may be inviable for the newly selected one 
                    if mybuttons['button_room'].isOver(pos):
                        i_selected = None 
                        j_selected = None
                        mybuttons['button_room'].active = True 
                        mybuttons['button_circulation'].active = False 
                        mybuttons['button_courtyard'].active = False 
                        mybuttons['button_staircase'].active = False
                        mybuttons['button_iwan'].active = False  
                        mybuttons['button_street'].active = False
                    if mybuttons['button_circulation'].isOver(pos):
                        i_selected = None 
                        j_selected = None
                        mybuttons['button_room'].active = False 
                        mybuttons['button_circulation'].active = True 
                        mybuttons['button_courtyard'].active = False 
                        mybuttons['button_staircase'].active = False 
                        mybuttons['button_iwan'].active = False 
                        mybuttons['button_street'].active = False
                    if mybuttons['button_courtyard'].isOver(pos):
                        i_selected = None 
                        j_selected = None
                        #rotate through the courtyard options indefinitely 
                        if mybuttons['button_courtyard'].active == True:
                            courtyard_choice += 1
                        courtyard_queued = courtyard_options[courtyard_choice % len(courtyard_options)]
                        mybuttons['button_courtyard'].text = f'{courtyard_queued.tag}'
                        mybuttons['button_room'].active = False 
                        mybuttons['button_circulation'].active = False 
                        mybuttons['button_courtyard'].active = True 
                        mybuttons['button_staircase'].active = False
                        mybuttons['button_iwan'].active = False 
                        mybuttons['button_street'].active = False
                    if mybuttons['button_staircase'].isOver(pos):
                        i_selected = None 
                        j_selected = None
                        mybuttons['button_room'].active = False 
                        mybuttons['button_circulation'].active = False 
                        mybuttons['button_courtyard'].active = False 
                        mybuttons['button_staircase'].active = True
                        mybuttons['button_iwan'].active = False
                        mybuttons['button_street'].active = False
                    if mybuttons['button_iwan'].isOver(pos):
                        i_selected = None 
                        j_selected = None
                        mybuttons['button_room'].active = False 
                        mybuttons['button_circulation'].active = False 
                        mybuttons['button_courtyard'].active = False 
                        mybuttons['button_staircase'].active = False 
                        mybuttons['button_iwan'].active = True 
                        mybuttons['button_street'].active = False
                    if mybuttons['button_street'].isOver(pos):
                        i_selected = None 
                        j_selected = None
                        mybuttons['button_room'].active = False 
                        mybuttons['button_circulation'].active = False 
                        mybuttons['button_courtyard'].active = False 
                        mybuttons['button_staircase'].active = False
                        mybuttons['button_iwan'].active = False 
                        mybuttons['button_street'].active = True

                    #determine which room is to be built with the build function
                    #the circulation type corresponds to the active zone 
                    if mybuttons['button_build'].isOver(pos):
                        if i_selected != None and j_selected != None: 
                            if mybuttons['button_room'].active == True: 
                                room = zone_order[0].build_order[0]
                            if mybuttons['button_circulation'].active == True: 
                                room = circulation_options[0]
                            if mybuttons['button_staircase'].active == True: 
                                room = myspaces['staircase']
                            if mybuttons['button_courtyard'].active == True: 
                                room = courtyard_queued
                            if mybuttons['button_iwan'].active == True: 
                                room = myspaces['iwan'] 
                            if mybuttons['button_street'].active == True: 
                                room = myspaces['street']

                            build(room, orientation, direction_ns, direction_we, i_selected, j_selected)
                            i_selected = None 
                            j_selected = None 

        #switch between ground and first floors and which lattices should be displayed 
        if mybuttons['button_bg'].active == True: 
            active_lattice = lattice_bg
            active_lattice_color = lattice_bg_color
            active_lattice_index = lattice_bg_index 
        if mybuttons['button_f1'].active == True: 
            active_lattice = lattice_f1
            active_lattice_color = lattice_f1_color
            active_lattice_index = lattice_f1_index 

        if len(zone_order) != 0: 
            if mybuttons['button_landscape'].active == True:
                orientation = 'landscape'
            if mybuttons['button_portrait'].active == True:
                orientation = 'portrait'
            if mybuttons['button_up'].active == True:
                direction_ns = 'up'
            if mybuttons['button_down'].active == True:
                direction_ns = 'down'
            if mybuttons['button_right'].active == True:
                direction_we = 'right'
            if mybuttons['button_left'].active == True:
                direction_we = 'left'

            #reset all unoccupied cells to white color 
            unoccupied = np.where(active_lattice == 0)
            active_lattice_color[unoccupied] = 0

            #make viable indices blink  
            delay = 500 
            current_time = pg.time.get_ticks()
            if (int(current_time / delay)) % 2 == 0: 
                tick_color = 0
            else: 
                tick_color = 24

            ###highlight the viable cells based on the allowed connection of the selected spatial unit 
            #the process for each spatial unit is the same, although for non-rooms there may be more than one connection possible (ex. staircases can connect to all circulation ID's)
            #neighbours returns a list of every cell with their corresponding neighbours based on the Von Neumann stencil 
            #s2_i finds all cells in the lattice that correspond to the ID of what the selected spatial unit is allowed to connect to 
            #now we have both a list of neighbours for each cell and the index of all allowed cells, which we can cross-check to make a list of cells which neighbour an allowed connection 
            #the final step is to filter out the allowed cells that are also unoccupied, resulting in a list if viable cells 
            viable = []
            if mybuttons['button_room'].active == True: 
                text_notification = f'Now building: {zone_order[0].build_order[0].tag}'
                neighbours = active_lattice.find_neighbours(stencil)
                s2_i = np.where((active_lattice==zone_order[0].build_order[0].connection).flatten())
            if mybuttons['button_circulation'].active == True: 
                text_notification = 'Now building: Circulation'
                neighbours = active_lattice.find_neighbours(stencil)
                s2_i = np.where((active_lattice==myspaces['circulation_general'].id).flatten()
                + (active_lattice==myspaces['circulation_emergency'].id).flatten() 
                + (active_lattice==myspaces['circulation_outpatient'].id).flatten()
                + (active_lattice==myspaces['circulation_inpatient'].id).flatten()
                + (active_lattice==myspaces['circulation_administration'].id).flatten()
                + (active_lattice==myspaces['staircase'].id).flatten())
            if mybuttons['button_courtyard'].active == True: 
                text_notification = f'Now building: {courtyard_queued.tag}'
                neighbours = active_lattice.find_neighbours(stencil)
                s2_i = np.where((active_lattice==myspaces['circulation_general'].id).flatten()
                + (active_lattice==myspaces['circulation_emergency'].id).flatten() 
                + (active_lattice==myspaces['circulation_outpatient'].id).flatten()
                + (active_lattice==myspaces['circulation_inpatient'].id).flatten()
                + (active_lattice==myspaces['circulation_administration'].id).flatten())
            if mybuttons['button_staircase'].active == True: 
                if mybuttons['button_bg'].active == True: 
                    text_notification = 'Now building: Staircase'
                    neighbours = active_lattice.find_neighbours(stencil)
                    s2_i = np.where((active_lattice==myspaces['circulation_general'].id).flatten()
                    + (active_lattice==myspaces['circulation_emergency'].id).flatten() 
                    + (active_lattice==myspaces['circulation_outpatient'].id).flatten()
                    + (active_lattice==myspaces['circulation_inpatient'].id).flatten()
                    + (active_lattice==myspaces['circulation_administration'].id).flatten())
                elif mybuttons['button_f1'].active == True: 
                    s2_i = []
            if mybuttons['button_iwan'].active == True: 
                text_notification = 'Now building: Iwan'
                neighbours = active_lattice.find_neighbours(stencil)
                s2_i = np.where((active_lattice==myspaces['circulation_general'].id).flatten()
                + (active_lattice==myspaces['circulation_emergency'].id).flatten() 
                + (active_lattice==myspaces['circulation_outpatient'].id).flatten()
                + (active_lattice==myspaces['circulation_inpatient'].id).flatten()
                + (active_lattice==myspaces['circulation_administration'].id).flatten())
            if mybuttons['button_street'].active == True: 
                text_notification = 'Now building: Street'
                neighbours = active_lattice.find_neighbours(stencil)
                s2_i = np.where((active_lattice==myspaces['street'].id).flatten() + (active_lattice==myspaces['parking'].id).flatten())
            s2_ni = neighbours[s2_i]
            s2_niu = np.unique(s2_ni.flatten())
            available_s2_niu = active_lattice.flatten()[s2_niu] == 0
            viable = s2_niu[available_s2_niu]    
            for i in range(len(viable)): 
                viable_i = np.where(active_lattice_index == viable[i])
                active_lattice_color[viable_i] = tick_color
        
            #color the selected cell purple 
            if i_selected != None and j_selected != None: 
                active_lattice_color[0,i_selected,j_selected] = 23

        #draw all game stuff like buttons, legends, etc. 
        draw_window(active_lattice, active_lattice_color, text_notification)

    pg.quit()


main()

After playing the game, the resulting configuration can be saved in an Excel sheet and exported to a Grasshopper script where the configuration can be properly analysed with their wide suite of analysis tools. 

In [9]:
def output_to_excel(): 
    output_list = []
    for i in range(lattice_rows):
        for j in range(lattice_cols):
            if np.all([0, 55, street.id, boundary.id, parking.id] != lattice_bg[0,i,j]):
                output_list.append([i,j,0])

            if np.all([0, 55, street.id, boundary.id, parking.id] != lattice_f1[0,i,j]):
                output_list.append([i,j,1])

    df_output = pd.DataFrame(output_list, columns=['x', 'y', 'z'])
    df_output.to_excel(os.path.join('Data', '1_Configuration_Output.xlsx'), sheet_name='configuration_output', index=False, header=True )


We added a starting point for evaluation tools in Python as well. This tool calculates the amount of circulation used for each zone and the entire floor plan. 

In [10]:
def evaluation_tools(): 

    ## Calculating circulation lengths 
    total_circulation = 0 
    list = [myspaces['circulation_general'], 
    myspaces['circulation_emergency'], 
    myspaces['circulation_outpatient'], 
    myspaces['circulation_inpatient'], 
    myspaces['circulation_administration']]
    for i in list: 
        circulation_length = len(np.where(lattice_bg == i.id)[0]) + len(np.where(lattice_f1 == i.id)[0])
        print(f'Total circulation used for {i.name} = {circulation_length}')
        total_circulation += circulation_length 
        print(f'Total circulation used for entire configuration = {total_circulation}')


Reflection

The game works and has a nice interface, much better than the first version where a player had to manually type in indices to select a cell in an array. There are some things that can be improved upon, both to improve the playability for the player (QoL) and to improve the functionality of the game itself. 

QoL changes would be regarding the building of rooms. It would be nice to display a preview of the room before it is built, because it is easy to make a mistake in orientation or direction. Also, keybinds should be relatively simple to implement to smoothen the build process. These are QoL improvements I would recommend future groups to work on. 

Regarding the functionality of the game, there are two major limitations. The first, which actually bothers me, is that spaces can not be aligned centered from the selected cell. It is not that this is impossible, but the way the build function is programmed with its complicated equations using u,v,q,r to make the syntax work would become even more complicated with this option, and since it is only an issue for rooms with a shape larger than 2 (which the vast majority of the program isnt) this issue ranked low on the long priority list. 

The second limitation is not so much a limitation on the playability of the game in its current form, but for further improvements. Right now the game does not recognize rooms as singular units (Python has no knowledge that two adjacent 2's are the same space). I have ideas on how to implement this, but probably would have required a second person to work on the game to make this work within the timeframe of the project, or possibly future students. 
Making this work would open up big doors to for example evaluate floor plans on room closeness, or to even start to computationally insert walls/doors/windows. This would have been the next thing to develop for me and I highly recommend to investigate this to future students! 

A final note, I had clean up a mess I made by inserting many global variables. I believe that I fully corrected that mistake (the game runs perfectly fine), but if any glitches occur please contact me and I'll be more than happy to correct the mistake. 